# Double Pendulum

The following equations were used to simulate a double pendulum (they are derived here: https://www.math24.net/double-pendulum/)  
Notation:  
$l_1$, $l_2$ - bar lengths  
$m_1$, $m_2$ - mass of balls  
$\alpha_1$, $\alpha_2$ - angular position (angles between bar and vertical)  
$p_1$, $p_2$ - momentum of balls  
$\dot{\alpha_1}$, $\dot{\alpha_2}$ - angular velocity (derivative of the angular position over time)  
$\dot{p_1}$, $\dot{p_2}$ - time derivative of momentum  
$g$ - gravitational acceleration

$$
\left\{\begin{matrix}
\begin{align*}
& \dot{\alpha_1} = \frac{p_1l_2-p_2l_1cos(\alpha_1-\alpha_2)}{l_1^2l_2(m_1+m_2sin^2(\alpha_1-\alpha_2))}
\\
\\ & \dot{\alpha_2} = \frac{p_2(m_1+m_2)l_1-p_1m_2l_2cos(\alpha_1-\alpha_2)}{m_2l_1l_2^2(m_1+m_2sin^2(\alpha_1-\alpha_2))}
\\
\\ & \dot{p_1} = -(m_1+m_2)gl_1sin(\alpha_1) - A_1 + A_2
\\
\\ & \dot{p_2} = -m_2gl_2sin(\alpha_2) + A_1 - A_2
\end{align*}
\end{matrix}\right.
$$

where:

$$
A_1 = \frac{p_1p_2sin(\alpha_1-\alpha_2)}{l_1l_2(m_1+m_2sin^2(\alpha_1-\alpha_2))}
$$

$$
A_2 = \frac{(p_1^2m_2l_2^2-2p_1p_2m_2l_1l_2cos(\alpha_1-\alpha_2)+p_2^2(m_1+m_2)l_1^2)sin(2(\alpha_1-\alpha_2))}{2l_1^2l_2^2(m_1+m_2sin^2(\alpha_1-\alpha_2))^2}
$$

The above system of equations can be written as

${\mathbf{Z}}' = \mathbf{f}(\mathbf{Z})$  

where  
$\mathbf{Z} = (\alpha_1, \alpha_2, p_1, p_2)^T$  
$\mathbf{f} = (f_1, f_2, f_3, f_4)^T$

The vector f corresponds to the functions on the right side of the system of equations.

These equations cannot be solved analytically. That's why we will calculate successive values numerically using the Runge-Kutta algorithm of the 4th order (RK4). This algorithm performs the following calculations:

$\mathbf{Z_{n+1}} = \mathbf{Z_n} + \frac{1}{6}(\mathbf{Y_1}+2\mathbf{Y_2}+2\mathbf{Y_3}+\mathbf{Y_4})$  

$\mathbf{Y_1} = h*\mathbf{f}(\mathbf{Z_n})$  
$\mathbf{Y_2} = h*\mathbf{f}(\mathbf{Z_n} + \frac{1}{2}\mathbf{Y_1})$  
$\mathbf{Y_3} = h*\mathbf{f}(\mathbf{Z_n} + \frac{1}{2}\mathbf{Y_2})$  
$\mathbf{Y_4} = h*\mathbf{f}(\mathbf{Z_n} + \mathbf{Y_3})$  

$h$ - step size

$Z_0$ are initial values.

### Video showing the effect 
https://www.youtube.com/watch?v=__Vkw7a9Ntk

## Implementation

In [1]:
from ipycanvas import Canvas, hold_canvas
from ipycanvas import MultiCanvas, hold_canvas
from ipywidgets import interact, IntSlider, FloatSlider
import ipywidgets as widgets
from threading import Thread
from time import sleep
import numpy as np

### Functions from the system of equations

In [2]:
def f1(l1,l2, m1,m2, alpha1,alpha2, p1,p2):
    return (p1*l2 - p2*l1*np.cos(alpha1-alpha2))/(l1*l1*l2*(m1+m2*(np.sin(alpha1-alpha2))**2))

def f2(l1,l2, m1,m2, alpha1,alpha2, p1,p2):
    return (p2*(m1+m2)*l1 - p1*m2*l2*np.cos(alpha1-alpha2))/(m2*l1*l2*l2*(m1+m2*(np.sin(alpha1-alpha2))**2))

def f3(l1,l2, m1,m2, alpha1,alpha2, p1,p2, g):
    a1 = (p1*p2*np.sin(alpha1-alpha2))/(l1*l2*(m1+m2*(np.sin(alpha1-alpha2))**2))
    a2 = ((p1*p1*m2*l2*l2 - 2*p1*p2*m2*l1*l2*np.cos(alpha1-alpha2) + p2*p2*(m1+m2)*l1*l1)*np.sin(2*(alpha1-alpha2)))/(2*l1*l1*l2*l2*(m1+m2*(np.sin(alpha1-alpha2))**2)**2)
    
    return -(m1+m2)*g*l1*np.sin(alpha1) - a1 + a2

def f4(l1,l2, m1,m2, alpha1,alpha2, p1,p2, g):
    a1 = (p1*p2*np.sin(alpha1-alpha2))/(l1*l2*(m1+m2*(np.sin(alpha1-alpha2))**2))
    a2 = ((p1*p1*m2*l2*l2 - 2*p1*p2*m2*l1*l2*np.cos(alpha1-alpha2) + p2*p2*(m1+m2)*l1*l1)*np.sin(2*(alpha1-alpha2)))/(2*l1*l1*l2*l2*(m1+m2*(np.sin(alpha1-alpha2))**2)**2)
    
    return -m2*g*l2*np.sin(alpha2) + a1 - a2


### Runge-Kutta 4th order algorithm (RK4)

In [3]:
def runge_kutta(zn, h, l1,l2, m1,m2, g):
    y1 = h*np.array([
        f1(l1,l2, m1,m2, zn[0], zn[1], zn[2], zn[3]),
        f2(l1,l2, m1,m2, zn[0], zn[1], zn[2], zn[3]),
        f3(l1,l2, m1,m2, zn[0], zn[1], zn[2], zn[3], g),
        f4(l1,l2, m1,m2, zn[0], zn[1], zn[2], zn[3], g)
    ])
    
    zn2 = zn + y1/2
    y2 = h*np.array([
        f1(l1,l2, m1,m2, zn2[0], zn2[1], zn2[2], zn2[3]),
        f2(l1,l2, m1,m2, zn2[0], zn2[1], zn2[2], zn2[3]),
        f3(l1,l2, m1,m2, zn2[0], zn2[1], zn2[2], zn2[3], g),
        f4(l1,l2, m1,m2, zn2[0], zn2[1], zn2[2], zn2[3], g)
    ])

    zn3 = zn + y2/2
    y3 = h*np.array([
        f1(l1,l2, m1,m2, zn3[0], zn3[1], zn3[2], zn3[3]),
        f2(l1,l2, m1,m2, zn3[0], zn3[1], zn3[2], zn3[3]),
        f3(l1,l2, m1,m2, zn3[0], zn3[1], zn3[2], zn3[3], g),
        f4(l1,l2, m1,m2, zn3[0], zn3[1], zn3[2], zn3[3], g)
    ])
    
    zn4 = zn + y3
    y4 = h*np.array([
        f1(l1,l2, m1,m2, zn4[0], zn4[1], zn4[2], zn4[3]),
        f2(l1,l2, m1,m2, zn4[0], zn4[1], zn4[2], zn4[3]),
        f3(l1,l2, m1,m2, zn4[0], zn4[1], zn4[2], zn4[3], g),
        f4(l1,l2, m1,m2, zn4[0], zn4[1], zn4[2], zn4[3], g)
    ])
    
    return zn + (y1+2*y2+2*y3+y4)/6

### Double pendulum class

In [4]:
class DoublePendulum():
    def __init__(self, canvas, l1,l2, m1,m2, alpha1,alpha2, p1,p2, g, h):
        self.canvas = canvas
        self.l1 = l1
        self.l2 = l2
        self.m1 = m1
        self.m2 = m2
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.p1 = p1
        self.p2 = p2
        self.g = g
        self.h = h
        
        self.r1 = np.sqrt(self.m1/np.pi) * 10
        self.r2 = np.sqrt(self.m2/np.pi) * 10
    
    def set_params(self, l1,l2, m1,m2, alpha1,alpha2, p1,p2, g, h):
        self.l1 = l1
        self.l2 = l2
        self.m1 = m1
        self.m2 = m2
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.p1 = p1
        self.p2 = p2
        self.g = g
        self.h = h
        
        self.r1 = np.sqrt(self.m1/np.pi) * 10
        self.r2 = np.sqrt(self.m2/np.pi) * 10
        
        
    def draw_background(self):
        canvas = self.canvas[0]
        canvas.fill_style = "black"
        canvas.fill_rect(0, 0, canvas.width, canvas.height)
    
    def draw_pendulum(self):
        dx = self.l1*np.sin(self.alpha1)
        dy = self.l1*np.cos(self.alpha1)

        dx2 = dx + self.l2*np.sin(self.alpha2)
        dy2 = dy + self.l2*np.cos(self.alpha2)
        
        canvas = self.canvas[2]
        with hold_canvas(canvas):
            canvas.clear()

            # lines
            canvas.stroke_style = "white"
            canvas.line_width = 2
            canvas.begin_path()
            canvas.move_to(canvas.width/2, canvas.height/2)
            canvas.line_to(canvas.width/2 + dx, canvas.height/2 + dy)
            canvas.line_to(canvas.width/2 + dx2, canvas.height/2 + dy2)
            canvas.stroke()
            
            # center point
            canvas.fill_style = "red"
            canvas.fill_arc(canvas.width/2, canvas.height/2, 5, 0, np.pi * 2)

            # circles
            canvas.fill_style = "green"
            canvas.begin_path()
            canvas.arc(canvas.width/2 + dx, canvas.height/2 + dy, self.r1, 0, np.pi * 2)
            canvas.arc(canvas.width/2 + dx2, canvas.height/2 + dy2, self.r2, 0, np.pi * 2)
            canvas.fill()
            
    def draw_trace(self):
        canvas = self.canvas[1]
        
        x = canvas.width/2 + self.l1*np.sin(self.alpha1) + self.l2*np.sin(self.alpha2)
        y = canvas.height/2 + self.l1*np.cos(self.alpha1) + self.l2*np.cos(self.alpha2)

        canvas.fill_style = "#dddddd"
        canvas.fill_arc(x, y, 1, 0, np.pi * 2)
        
    def update(self):
        zn = np.array([
            self.alpha1,
            self.alpha2,
            self.p1,
            self.p2
        ], dtype="double")
        
        zn_new = runge_kutta(zn, self.h, self.l1,self.l2, self.m1,self.m2, self.g)
        
        self.alpha1 = zn_new[0] % (2*np.pi)
        self.alpha2 = zn_new[1] % (2*np.pi)
        self.p1 = zn_new[2]
        self.p2 = zn_new[3]
        

### Animation thread class

In [5]:
class DoublePendulumThread(Thread):
    def __init__(self, double_pendulum):
        self.dp = double_pendulum
        self.running = True
        super(DoublePendulumThread, self).__init__()
        self.dp.draw_background()
    
    def stop(self):
        self.running = False
        
    def run(self):
        while self.running:
            self.dp.draw_trace()
            self.dp.draw_pendulum()
            self.dp.update()
            sleep(0.01)

### Handling clicking on animation

In [6]:
def handle_mouse_down(x, y):
    global dpt
    if dpt.running:
        dpt.stop()
    else:
        dpt = DoublePendulumThread(dp2)
        dpt.start()

### Standard parameters

In [7]:
l1 = 100 # m
l2 = 160 # m
m1 = 5   # kg
m2 = 10   # kg
alpha1 = np.pi/2 # rad
alpha2 = np.pi/3 # rad
p1 = 0   # kg*m/s
p2 = 0   # kg*m/s
g = 9.81 # m/s^2
h = 0.1

### Selecting parameters

In [8]:
canvas = MultiCanvas(3, width=1000, height=800)
dp = DoublePendulum(canvas, l1,l2, m1,m2, alpha1,alpha2, p1,p2, g,h)
dp.draw_background()

@interact(
    length_1 = IntSlider(min=10, max=180, step=10, value=l1),
    length_2 = IntSlider(min=10, max=180, step=10, value=l2),
    mass_1 = FloatSlider(min=0.1, max=30.01, step=0.1, value=m1),
    mass_2 = FloatSlider(min=0.1, max=30.01, step=0.1, value=m2),
    alpha_1 = FloatSlider(min=0, max=2*np.pi, step=0.01, value=alpha1),
    alpha_2 = FloatSlider(min=0, max=2*np.pi, step=0.01, value=alpha2),
    momentum_1 = IntSlider(min=0, max=100, step=1, value=p1),
    momentum_2 = IntSlider(min=0, max=100, step=1, value=p2),
    gravity = FloatSlider(min=-30, max=30, step=0.01, value=g),
    h_time = FloatSlider(min=0.01, max=1, step=0.01, value=h)
)
def set_params(length_1,length_2, mass_1,mass_2, alpha_1,alpha_2, momentum_1,momentum_2, gravity, h_time):
    global l1, l2, m1, m2, alpha1, alpha2, p1, p2, g, h, dp
    l1 = length_1
    l2 = length_2
    m1 = mass_1
    m2 = mass_2
    alpha1 = alpha_1
    alpha2 = alpha_2
    p1 = momentum_1
    p2 = momentum_2
    g = gravity
    h = h_time
    
    dp.set_params(length_1,length_2, mass_1,mass_2, alpha_1,alpha_2, momentum_1,momentum_2, gravity,h_time)
    dp.draw_pendulum()
    
canvas

interactive(children=(IntSlider(value=100, description='length_1', max=180, min=10, step=10), IntSlider(value=…

MultiCanvas(height=800, width=1000)

### Simulation
To start simulation run the cell below.  
To stop or resume it click on the animation.

In [9]:
canvas[1].clear()
canvas[2].on_mouse_down(handle_mouse_down)

dp2 = DoublePendulum(canvas, l1,l2, m1,m2, alpha1,alpha2, p1,p2, g,h)

# stop prev thread
try:
    dpt
except NameError:
    pass
else:
    dpt.stop()
    del dpt

dpt = DoublePendulumThread(dp2)
dpt.start()

### Stopping simulation
Simulation can be stopped by clicking on the animation or by running the cell below

In [10]:
# stop by running this cell
# you can also pause by clicking on canvas
dpt.stop()